In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.005058765411376953
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 20.713709487431444
200000 33.178178192776365
300000 33.50657186295394
400000 33.497802378748744
500000 33.54567893041398
600000 33.14411885506286
700000 35.06756898074096
800000 33.24322445691142
900000 32.53749378340247
1000000 32.86705739559905
1100000 32.18937949687051
1200000 32.641898144573375
1300000 15.721546389563118
1400000 23.932723524986176
1500000 32.065656395647075
1600000 32.38467876144721
1700000 32.12870691978546
1800000 32.221011074839886
1900000 32.54881015413616
2000000 34.05769143243074
2100000 32.34918855937817
2200000 32.269599922843504
2300000 32.40186316098954
2400000 32.96516201604668
2500000 32.528555163790706
2600000 12.864133313908425
2700000 28.032530967347864
2800000 41.22764609969224
2900000 41.5206382951685
3000000 40.631191

29700000 27.99421394715422
29800000 29.899680168359954
29900000 31.62005183434222
30000000 31.978543036788803
30100000 31.60141849452547
30200000 31.251136941228687
30300000 32.33676371229989
30400000 33.72042777293977
30500000 32.66782577596705
30600000 31.484425754176517
30700000 12.776739950785863
30800000 34.39033656013962
30900000 34.16910966574875
31000000 33.564799586422936
31100000 20.272714472967728
31200000 24.761200011033342
31300000 33.64972854697265
31400000 33.56388239482392
31500000 33.32496885448038
31600000 32.372031039091915
31700000 34.323859137066044
31800000 33.07699371112175
31900000 31.895628778265564
32000000 32.217715864025465
32100000 30.73801276873047
32200000 7.394932703363567
32300000 32.23452073080163
32400000 31.602965372200526
32500000 32.032990787505234
32600000 32.91994044053895
32700000 32.68072250528236
32800000 31.493114231017554
32900000 32.88864440541498
33000000 32.60832527152133
33100000 32.81387370687881
33200000 20.33689638696257
33300000 24.8

59500000 29.636003689068
59600000 15.582252114333327
59700000 23.32625460297575
59800000 28.93602052578418
59900000 27.492379014994324
60000000 27.66509234642307
60100000 26.81071992706096
60200000 26.397136549039182
60300000 26.048782385846305
60400000 22.35157848390366
60500000 11.419348417605896
60600000 25.74610220145168
60700000 26.190100667534725
60800000 26.40037757937177
60900000 27.094903006980374
61000000 27.446795860581076
61100000 27.576818938195903
61200000 27.76922240683856
61300000 13.433073581364532
61400000 26.857374487506874
61500000 29.126649443911607
61600000 28.19788185469789
61700000 29.678327581457136
61800000 29.506061653477854
61900000 29.75040510125956
62000000 30.28851803098737
62100000 30.133390706725816
62200000 14.154385368740842
62300000 25.93032294395098
62400000 29.239714495156218
62500000 28.32122627413203
62600000 29.696179965916436
62700000 28.01312663311437
62800000 27.274863602737227
62900000 27.562287628375277
63000000 16.468241223747206
63100000 

89200000 23.335363528868513
89300000 24.38502297187537
89400000 8.384868057599007
89500000 26.18507215998131
89600000 25.218697135565648
89700000 25.950749435158983
89800000 25.961632482636812
89900000 25.90765523585414
90000000 27.200985257870407
90100000 25.195097071182616
90200000 26.236050256606188
90300000 26.15486326502321
90400000 25.448621258355885
90500000 25.24399511390498
90600000 25.285783547524527
90700000 9.691241352184
90800000 22.232192607446052
90900000 23.619762487198283
91000000 22.5696193110641
91100000 23.297058945690278
91200000 23.08867012851198
91300000 10.917066972321505
91400000 18.331749632227464
91500000 23.382388162101673
91600000 23.01706103423363
91700000 22.918010607649553
91800000 23.010152734342324
91900000 16.234906283503744
92000000 17.766080105310568
92100000 24.413602317609325
92200000 24.701258116377364
92300000 25.768929767211098
92400000 24.993836927273716
92500000 26.07301474969059
92600000 25.124514239421906
92700000 26.395233085133288
9280000

118300000 23.147166151891202
118400000 24.67831525624111
118500000 23.863248390648273
118600000 24.495198426914552
118700000 24.12653019330358
118800000 23.083641418127073
118900000 22.589264582414167
119000000 21.628627568479676
119100000 22.322514311569385
119200000 6.7013902089821755
119300000 21.45545763750729
119400000 22.05754959865266
119500000 21.365253857228183
119600000 7.7067951028292265
119700000 19.97326643235246
119800000 21.61119255544495
119900000 21.740482285633885
120000000 12.095259655363783
120100000 18.368941703605465
120200000 21.85346354227545
120300000 21.387324073184775
120400000 22.188201827175757
120500000 22.316165809198818
120600000 23.14739252989009
120700000 23.64717860662074
120800000 23.357627105788513
120900000 23.542313927073547
121000000 23.110149855108364
121100000 22.662113749667196
121200000 22.484564002383514
121300000 22.321652782785517
121400000 20.9835038307453
121500000 21.258938556552263
121600000 20.35432149106251
121700000 21.0257885114909

...

232900000 10.74682534029875
233000000 11.277242620560195
233100000 8.960620981748264
233200000 5.560700605233834
233300000 11.00205362534079
233400000 10.958877663121156
233500000 11.445448632470816
233600000 11.519064039724142
233700000 11.708133379399763
233800000 12.073465098581059
233900000 12.00836287218597
234000000 12.109534220913071
234100000 11.693160163823192
234200000 11.652555200047246
234300000 11.723775471136863
234400000 11.52066259270314
234500000 10.881841526081278
234600000 10.925993985068493
234700000 10.468205912413536
234800000 11.059882395740354
234900000 3.875984467001033
235000000 11.048519575009417
235100000 10.761563358432491
235200000 10.860981086438434
235300000 4.604741695448768
235400000 8.97439448886594
235500000 10.830948761993932
235600000 10.560857874792731
235700000 8.241373105481669
235800000 6.473668147414593
235900000 10.462868614199872
236000000 10.864477814877597
236100000 11.380040211803038
236200000 11.266981694872142
236300000 11.7827248109423

261900000 9.060099573138432
262000000 9.240959740139454
262100000 9.247987304621306
262200000 9.833398947965984
262300000 8.8342696660649
262400000 9.1839151564922
262500000 9.246082194657097
262600000 9.244437726684373
262700000 9.129933586716843
262800000 8.913969039821549
262900000 7.70860737818831
263000000 4.518584133408888
263100000 8.315929157375505
263200000 8.388164821953588
263300000 8.293152399493636
263400000 8.33105481743137
263500000 8.605351006474832
263600000 2.4265988172069837
263700000 8.189819309961184
263800000 8.550731225630834
263900000 8.621441487952033
264000000 8.45539690792875
264100000 8.60149017315256
264200000 5.76159180201407
264300000 5.882195330721086
264400000 9.04521747224074
264500000 9.093651565024516
264600000 9.300767392200077
264700000 9.251869766212504
264800000 9.563414608415771
264900000 9.724207434601327
265000000 9.349826160368757
265100000 8.901519392238741
265200000 9.146169124680378
265300000 8.813806980852414
265400000 8.727655443430228
2

291200000 6.4896229695728245
291300000 5.827497109004773
291400000 6.2774737595672345
291500000 5.778492167358893
291600000 6.053560589319178
291700000 6.086072150648841
291800000 5.788447727984894
291900000 1.1090085834200265
292000000 6.2715198932400344
292100000 5.649680693106036
292200000 6.032665213047067
292300000 5.87085102558881
292400000 6.175776542527876
292500000 6.267442252810803
292600000 5.9946530395083295
292700000 4.244392562327321
292800000 4.470930882473701
292900000 6.650723835649932
293000000 6.095002995561512
293100000 6.6914382905187155
293200000 6.6488551878398345
293300000 6.205593909195338
293400000 6.59384538561524
293500000 6.549479302247523
293600000 2.1042145553846536
293700000 6.518762371092186
293800000 6.425650117039871
293900000 6.213838324504541
294000000 5.880053248953866
294100000 6.003467906424432
294200000 5.5343910782966645
294300000 5.921649065708409
294400000 4.41167297575291
294500000 3.126205721254968
294600000 5.479180590562418
294700000 5.94

320300000 3.3654652928957223
320400000 3.380828458959919
320500000 3.531655048381142
320600000 3.550824340241489
320700000 3.646672886581467
320800000 3.45557963533209
320900000 3.5276923676224348
321000000 3.5965317867537983
321100000 3.5696504040189665
321200000 1.9225562931648996
321300000 2.7864407183985667
321400000 3.637663087874977
321500000 3.469807970670938
321600000 3.166146401464187
321700000 1.4810962651498736
321800000 3.3522328357438838
321900000 3.3143643715682662
322000000 3.405098151193513
322100000 3.2355184437551863
322200000 3.18716123644139
322300000 3.312523011559518
322400000 3.2060442288252453
322500000 3.1206100092903406
322600000 3.065507390119904
322700000 2.0520926306002147
322800000 1.9124934763785242
322900000 3.024315934632347
323000000 3.2077281441077887
323100000 3.154126510803348
323200000 3.1940928401034574
323300000 3.12312539949345
323400000 3.12172839200714
323500000 3.25931197432735
323600000 3.2199744798906007
323700000 3.243172319727519
32380000

348900000 0.652582998008536
349000000 0.6294471104914209
349100000 0.6530443099603057
349200000 0.6612001437263946
349300000 0.6082812849272926
349400000 0.6372562185809632
349500000 0.6300458965014577
349600000 0.5872329358163954
349700000 0.2964158271293978
349800000 0.27083820467915204
349900000 0.4752264700204869
350000000 0.4334595637189149
350100000 0.45533912594454284
350200000 0.4072423229608933
350300000 0.43242183452392746
350400000 0.41870737092173965
350500000 0.41314036045156716
350600000 0.3905038667000592
350700000 0.3786135365618951
350800000 0.3884891202385545
350900000 0.3432456000034935
351000000 0.18881337149687144
351100000 0.2583733868946214
351200000 0.3215834995460345
351300000 0.3396842180587398
351400000 0.32027003643670077
351500000 0.30671423640071216
351600000 0.3136095775960485
351700000 0.2958475029169321
351800000 0.2948163726731744
351900000 0.26175808383093807
352000000 0.2765071892884592
352100000 0.24857284752976763
352200000 0.2468962815742751
best 